In [1]:
import numpy
from numpy.random import randn

from openfermion.transforms._jordan_wigner import *

from openfermion.utils._trotter_exp_to_qgates import *

from openfermion.utils import (commutator, count_qubits, expectation,
                               hermitian_conjugated, normal_ordered,
                               jordan_wigner_sparse, jw_hartree_fock_state,
                               s_squared_operator, sz_operator)

from openfermion.utils._unitary_cc import (uccsd_generator,
                                           uccsd_singlet_generator,
                                           uccsd_singlet_get_packed_amplitudes,
                                           uccsd_singlet_paramsize)

In [2]:
test_orbitals = 4

single_amplitudes = randn(*(test_orbitals,) * 2)
double_amplitudes = randn(*(test_orbitals,) * 4)
print('singles')
print(single_amplitudes)
print('doubles')
print(double_amplitudes)


singles
[[ 0.09009994  0.40754339 -1.08380695  0.70262557]
 [-1.48254042 -1.11621033  0.59799343 -1.13931414]
 [ 0.15214072  1.06951274  0.05589617 -0.69964846]
 [-0.6199874  -0.99570951  2.72384957  0.85881463]]
doubles
[[[[ 1.87288155 -0.34344984  0.98661725 -2.66011485]
   [-0.04931829  0.28893147 -0.93395842 -1.90513569]
   [ 1.17418463 -0.76884108 -0.43092618  1.10071658]
   [ 0.53376827  0.39940258  0.243249   -1.45040928]]

  [[-1.17589011  0.6747305   0.33153821 -0.55113445]
   [ 0.38532845  1.15431216 -0.7894676  -0.02392574]
   [ 1.06738982  1.10713296  0.21337542 -1.70322367]
   [-0.75070551 -1.26225539 -0.51996789  2.03465843]]

  [[-1.28690256 -0.78034097 -1.02322325  1.0602987 ]
   [-1.74667292 -0.93337424  1.47779667  0.00662061]
   [ 0.16924367 -0.42002913  0.80126676 -1.26587317]
   [ 0.75788705 -0.26793323  0.03897668 -0.14752336]]

  [[-0.28619285  0.71380602  0.10502253  1.45146979]
   [ 0.51064786 -0.12905344  0.92018661 -0.83926671]
   [ 0.358246    0.57369758 -0.

In [3]:
generator = uccsd_generator(single_amplitudes, double_amplitudes)


qubit_op = jordan_wigner(generator)

#of_generator = trotter_operator_grouping(qubit_op)

# print('openfermion trotterized cluster operator')
# print(of_generator)
# print('')

qubit_op *= 1.0j

print('openferion pre trotterized generator')
print(generator)
print('')
print(qubit_op)

openferion pre trotterized generator
0.22139362996548462 [0^ 0 0^ 1] +
0.15338433593892387 [0^ 0 0^ 2] +
-2.7593434945801274 [0^ 0 0^ 3] +
1.1265718199271808 [0^ 0 1^ 0] +
-0.079357601367684 [0^ 0 1^ 1] +
-1.884316708890315 [0^ 0 1^ 2] +
-2.3983656355262397 [0^ 0 1^ 3] +
2.4610871959243408 [0^ 0 2^ 0] +
-1.2793367964405145 [0^ 0 2^ 1] +
0.29800410168940467 [0^ 0 2^ 2] +
2.902877432453213 [0^ 0 2^ 3] +
0.8199611241575533 [0^ 0 3^ 0] +
1.864113588370233 [0^ 0 3^ 1] +
-0.43807588937462566 [0^ 0 3^ 2] +
0.04398583910863518 [0^ 0 3^ 3] +
1.8900838109201392 [0^ 1] +
-1.1265718199271808 [0^ 1 0^ 0] +
-0.7663712909186117 [0^ 1 0^ 1] +
-0.6415006187311478 [0^ 1 0^ 2] +
0.9073288899544467 [0^ 1 0^ 3] +
0.10558221778633525 [0^ 1 1^ 1] +
-1.4625183369861432 [0^ 1 1^ 2] +
1.0190107480867137 [0^ 1 1^ 3] +
2.8140627423181925 [0^ 1 2^ 0] +
0.31217037116721646 [0^ 1 2^ 1] +
1.062379664572759 [0^ 1 2^ 2] +
-2.3746360595304346 [0^ 1 2^ 3] +
-1.2613533714377847 [0^ 1 3^ 0] +
-3.0155885552895447 [0^ 1 3^ 1

In [4]:
from forestopenfermion.pyquil_connector import qubitop_to_pyquilpauli
from openfermion.ops import QubitOperator


In [5]:
# pauli_sum_representation = qubitop_to_pyquilpauli(qubit_op)
# print(pauli_sum_representation)

In [6]:
from pyquil.quil import Program
from forestopenfermion import exponentiate
from pyquil.gates import *

pyquil_program = exponentiate(qubit_op)
pyquil_program = Program(X(0), X(1), pyquil_program)
print(pyquil_program)
# print(len(pyquil_program))

X 0
X 1
RX(pi/2) 0
H 1
CNOT 0 1
RZ(3.5715648633438546) 1
CNOT 0 1
RX(-pi/2) 0
H 1
H 0
RX(pi/2) 1
CNOT 0 1
RZ(-3.5715648633438546) 1
CNOT 0 1
H 0
RX(-pi/2) 1
RX(pi/2) 0
H 2
CNOT 0 1
CNOT 1 2
RZ(-2.6659321885680383) 2
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 2
H 0
RX(pi/2) 2
CNOT 0 1
CNOT 1 2
RZ(2.6659321885680383) 2
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 2
RX(pi/2) 0
H 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(2.1207810104585363) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-pi/2) 0
H 3
H 0
RX(pi/2) 3
CNOT 0 1
CNOT 1 2
CNOT 2 3
RZ(-2.1207810104585363) 3
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
RX(-pi/2) 3
RX(pi/2) 1
H 2
CNOT 1 2
RZ(-1.6251561074120002) 2
CNOT 1 2
RX(-pi/2) 1
H 2
H 1
RX(pi/2) 2
CNOT 1 2
RZ(1.6251561074120002) 2
CNOT 1 2
H 1
RX(-pi/2) 2
RX(pi/2) 1
H 3
CNOT 1 2
CNOT 2 3
RZ(-3.3315024691868658) 3
CNOT 2 3
CNOT 1 2
RX(-pi/2) 1
H 3
H 1
RX(pi/2) 3
CNOT 1 2
CNOT 2 3
RZ(3.3315024691868658) 3
CNOT 2 3
CNOT 1 2
H 1
RX(-pi/2) 3
RX(pi/2) 2
H 3
CNOT 2 3
RZ(-4.681688258786461) 3
CNOT 2 3
RX(-pi/2) 2
H 3
H 2
RX(pi/2) 3
CNOT 2 3
RZ(4.

In [7]:
from pyquil.api import WavefunctionSimulator
from pyquil import Program


print('')
wfn = WavefunctionSimulator().wavefunction(pyquil_program)
print(wfn)


(0.0508605197+0j)|0011> + (0.3590020859+0j)|0101> + (-0.3848101419+0j)|0110> + (0.3960729215+0j)|1001> + (0.6007362238+0j)|1010> + (0.4502156093+0j)|1100>


# The correct answer!

In [8]:
# (0.2671478502+0j)|01> + (0.9636555537+0j)|10>

In [9]:
#[H0  Rx1  cX0-1  Rz1  cX0-1  H0  Rx1  Rx0  H1  cX0-1  Rz1  cX0-1  Rx0  H1]

# Now begin importing trial circuit to Qforte

In [10]:
import qforte

In [11]:
#get from openferion
qforte_generator = qforte.build_from_openfermion(qubit_op)
qforte.smart_print(qforte_generator)


 Quantum operator:
(0.8399478291489243+0j)[X0 X1 X2 Y3]
+ (-0.23533324298393435+0j)[X0 X1 Y2 X3]
+ (-1.7857824316719273+0j)[X0 Y1]
+ (-0.44635235175390586+0j)[X0 Y1 X2 X3]
+ (0.15826223441108392+0j)[X0 Y1 Y2 Y3]
+ (0.30469553170127484+0j)[X0 Y1 Z2]
+ (0.21049179674904506+0j)[X0 Y1 Z3]
+ (1.3329660942840191+0j)[X0 Z1 Y2]
+ (1.4981841087439554+0j)[X0 Z1 Y2 Z3]
+ (-1.0603905052292681+0j)[X0 Z1 Z2 Y3]
+ (1.1331084164585978+0j)[X0 Z1 Y3]
+ (-0.1462493003209332+0j)[X0 Y2]
+ (-0.7275180076902948+0j)[X0 Z2 Y3]
+ (1.7857824316719273+0j)[Y0 X1]
+ (-0.15826223441108392+0j)[Y0 X1 X2 X3]
+ (0.44635235175390586+0j)[Y0 X1 Y2 Y3]
+ (-0.30469553170127484+0j)[Y0 X1 Z2]
+ (-0.21049179674904506+0j)[Y0 X1 Z3]
+ (0.23533324298393435+0j)[Y0 Y1 X2 Y3]
+ (-0.8399478291489243+0j)[Y0 Y1 Y2 X3]
+ (-1.3329660942840191+0j)[Y0 Z1 X2]
+ (-1.4981841087439554+0j)[Y0 Z1 X2 Z3]
+ (1.0603905052292681+0j)[Y0 Z1 Z2 X3]
+ (-1.1331084164585978+0j)[Y0 Z1 X3]
+ (0.1462493003209332+0j)[Y0 X2]
+ (0.7275180076902948+0j)[Y0 Z2 X3]

# Now try to exponentiate the full operator 

In [12]:
expn = qforte.trotterization.trotterize(qforte_generator)
qforte.smart_print(expn)
#expn.str()


 Quantum circuit:
[H0  H1  H2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  H2  Rx3  H0  H1  Rx2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  H1  Rx2  H3  H0  Rx1  cX1-0  Rz1  cX1-0  H0  Rx1  H0  Rx1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  H2  H3  H0  Rx1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx1  Rx2  Rx3  H0  Rx1  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  H0  Rx1  H0  Rx1  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  H0  Rx1  H0  Rx2  cX1-0  cX2-1  Rz2  cX2-1  cX1-0  H0  Rx2  H0  Rx2  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx2  H0  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  H0  Rx3  H0  Rx3  cX1-0  cX3-1  Rz3  cX3-1  cX1-0  H0  Rx3  H0  Rx2  cX2-0  Rz2  cX2-0  H0  Rx2  H0  Rx3  cX2-0  cX3-2  Rz3  cX3-2  cX2-0  H0  Rx3  Rx0  H1  cX1-0  Rz1  cX1-0  Rx0  H1  Rx0  H1  H2  H3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  H2  H3  Rx0  H1  Rx2  Rx3  cX1-0  cX2-1  cX3-2  Rz3  cX3-2  cX2-1  cX1-0  Rx0  H1  

In [18]:
qc2 = qforte.QuantumComputer(4)
qc2.apply_gate(qforte.make_gate('X',0,0))
qc2.apply_gate(qforte.make_gate('X',1,1))
qc2.str()


['(0.000000 +0.000000 i) |0000>',
 '(0.000000 +0.000000 i) |1000>',
 '(0.000000 +0.000000 i) |0100>',
 '(1.000000 +0.000000 i) |1100>',
 '(0.000000 +0.000000 i) |0010>',
 '(0.000000 +0.000000 i) |1010>',
 '(0.000000 +0.000000 i) |0110>',
 '(0.000000 +0.000000 i) |1110>',
 '(0.000000 +0.000000 i) |0001>',
 '(0.000000 +0.000000 i) |1001>',
 '(0.000000 +0.000000 i) |0101>',
 '(0.000000 +0.000000 i) |1101>',
 '(0.000000 +0.000000 i) |0011>',
 '(0.000000 +0.000000 i) |1011>',
 '(0.000000 +0.000000 i) |0111>',
 '(0.000000 +0.000000 i) |1111>']

In [19]:
qc2.apply_circuit(expn)
qc2.str()

['(0.090537 +0.000000 i) |0000>',
 '(0.000000 -0.000000 i) |1000>',
 '(0.000000 +0.000000 i) |0100>',
 '(0.107876 +0.000000 i) |1100>',
 '(0.000000 +0.000000 i) |0010>',
 '(0.313677 +0.000000 i) |1010>',
 '(-0.611901 +0.000000 i) |0110>',
 '(0.000000 +0.000000 i) |1110>',
 '(0.000000 +0.000000 i) |0001>',
 '(-0.234492 +0.000000 i) |1001>',
 '(0.066284 +0.000000 i) |0101>',
 '(0.000000 +0.000000 i) |1101>',
 '(-0.616988 +0.000000 i) |0011>',
 '(0.000000 +0.000000 i) |1011>',
 '(0.000000 +0.000000 i) |0111>',
 '(0.259415 +0.000000 i) |1111>']

In [15]:
#(0.3510953756+0j)|0011> + (-0.6045891379+0j)|0101> + (-0.1071633537+0j)|0110> + (0.1697168339+0j)|1001> + (-0.6312331758+0j)|1010> + (-0.2691856261+0j)|1100>